In [1]:
# clear workspace
rm(list=ls())

# load libraries
library(readr)
library(dplyr)
library(magrittr)
library(ggplot2)
library(lfe)
library(stargazer)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: Matrix

Please cite as: 

 Hlavac, Marek (2015). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2. http://CRAN.R-project.org/package=stargazer 



In [2]:
# set absolute directory to be source file directory
# working_dir <-
#   rstudioapi::getActiveDocumentContext()$path %>%
#   dirname()
# setwd(working_dir)
working_dir <- getwd()

# define paths
IN_DIR <- "data/GiveDirectly/merged.csv"
OUT_DIR <- "data/GiveDirectly/Output/"

# load data
df <- read_csv(IN_DIR)

Parsed with column specification:
cols(
  .default = col_double(),
  s1_hhid_key = col_character(),
  s6_q1a_electype = col_character()
)
See spec(...) for full column specifications.


In [3]:
names(df)

[1] "s1_hhid_key"               "s6_q1_haselectricity"     
 [3] "s6_q1a_electype"           "s6_q6_acresowned"         
 [5] "s6_q6b_agacresowned"       "s6_q7_rentedland"         
 [7] "s6_q7a_acresrented"        "s6_q7b_monthsrented"      
 [9] "s6_q7ci_landrent"          "s6_q8_landrenting"        
[11] "s6_q8a_acresrenting"       "s6_q8b_agacresrenting"    
[13] "s6_q8c_monthsrenting"      "s19_gps_latitude"         
[15] "s19_gps_longitude"         "treat"                    
[17] "hi_sat"                    "p1_assets_wins"           
[19] "p1_6_nonthatchedroof"      "net_asset_value_home_wins"
[21] "p2_consumption_wins"       "p3_1_agprofit_wins"       
[23] "land_agacresowned_BL"      "land_agvalue_BL"          
[25] "aglandrentalcost_BL"       "aglanduse_BL"             
[27] "p1_assets_wins_BL"         "net_asset_value_home_BL"  
[29] "p1_6_nonthatchedroof_BL"   "satcluster"               
[31] "all_sat_house"             "colored_sat_house"        
[33] "metal_sat_house"           "thatched_sat_house"       
[35] "all_sat_size_mean"         "colored_sat_size_mean"    
[37] "metal_sat_size_mean"       "thatched_sat_size_mean"   
[39] "all_sat_size_sum"          "colored_sat_size_sum"     
[41] "metal_sat_size_sum"        "thatched_sat_size_sum"    
[43] "nightlight"

In [4]:
df <- df %>% subset(p1_6_nonthatchedroof_BL == 0 & (!is.na(treat)))

In [5]:
df[is.na(df$all_sat_size_mean),]$all_sat_size_mean = 0

In [6]:
reg1 <- felm(nightlight ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg2 <- felm(all_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg3 <- felm(all_sat_size_mean ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg4 <- felm(thatched_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df)
reg5 <- felm(metal_sat_house ~ treat + hi_sat | 0 | 0 | satcluster, df)

In [7]:
df_control <- df %>% subset(treat == 0 & hi_sat == 0)
outcomes <- c("nightlight", "all_sat_house", "all_sat_size_mean", "thatched_sat_house", "metal_sat_house")
means <- colMeans(df_control[outcomes]) %>% as.vector() %>% round(3)
stds <- apply(df_control[outcomes], 2, sd) %>% as.vector() %>% round(3) %>%
    lapply(function(x) paste0("(", x, ")")) %>% unlist()

In [8]:
output <- stargazer(reg1, reg2, reg3, reg4, reg5,
                    dep.var.labels=c("Night Lights",
                                     "No. of Houses within 100m Distance",
                                     "Mean Size of Houses (sq meters)",
                                     "No. of Houses w/ Thatched Roof",
                                     "No. of Houses w/ Metal Roof"),
                    keep=c("treat"),
                    keep.stat=c("n"),
                    add.lines=list(c("Control Mean", means), c("(Std. Dev.)", stds)),
                    covariate.labels=c("Treatment", "Control Mean"))
cat(paste0(output, "\n"), file=paste0(OUT_DIR, "abbrev.tex"))


% Table created by stargazer v.5.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Sat, Dec 07, 2019 - 03:07:06 AM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lccccc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{5}{c}{\textit{Dependent variable:}} \\ 
\cline{2-6} 
\\[-1.8ex] & Night Lights & No. of Houses within 100m Distance & Mean Size of Houses (sq meters) & No. of Houses w/ Thatched Roof & No. of Houses w/ Metal Roof \\ 
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5)\\ 
\hline \\[-1.8ex] 
 Treatment & 0.011 & 0.342$^{***}$ & 8.860$^{***}$ & $-$0.263$^{***}$ & 0.602$^{***}$ \\ 
  & (0.070) & (0.106) & (3.185) & (0.065) & (0.098) \\ 
  & & & & & \\ 
\hline \\[-1.8ex] 
Control Mean & 0.485 & 3.913 & 167.602 & 1.665 & 2.167 \\ 
(Std. Dev.) & (2.002) & (2.845) & (117.475) & (1.627) & (2.262) \\ 
Observations & 4,529 & 4,529 & 4,529 & 4,529 & 4,529 \\ 
\hline 
\hline \\[-1.8ex] 
\textit{Note:} 